- Cliente
    - variáveis:
        * Recencia: O quão recente é o último acesso
        * Frequências: Quantas vezes ele acessou a plataforma
        * Cadastro: Quando ele se cadastrou 
    - Com essas 3 variáveis , quero dizer se meu cliente está: Alegre(2) / Neutro(1) / Insatisfeito(0) com meu produto! (3 CATEGORIAS) 

- Assim, vamos utilizar o ONE VS REST para classifcação com mais de 2 categorias:
    * 0=>0      1=>1,2  LinearSVC 0 ou do resto (38%, resto 62%) 
    * 0=>0,2    1=>1    LinearSVC 1 ou do resto (44%, resto 56%) 
    * 0=>0,1    2=>2    LinearSVC 2 ou do resto (20%, resto 80%) 

* Também podemos utilizar o ONE VS ONE para classifcação com mais de 2 categorias tabém:
    * se tivermos n categorias, vamos rodar n*n classificadores. Onde temos um a um cada categoria 

In [3]:
from typing import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score # já vai fazar o fit, predict e devolver as notas

df = pd.read_csv('clientes.csv')
X_df = df[['recencia','frequencia', 'semanas_de_inscricao']]
Y_df = df['situacao']

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values 

porcentagem_de_treino_e_teste = 0.9

tamanho_de_treino_e_teste = int(porcentagem_de_treino_e_teste * len(Y))

treino_dados = X[:tamanho_de_treino_e_teste]
treino_marcacoes = Y[:tamanho_de_treino_e_teste]

validacao_dados = X[tamanho_de_treino_e_teste:]
validacao_marcacoes = Y[tamanho_de_treino_e_teste:]

In [4]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acerto = np.mean(scores)
    msg = f'A taxa de acerto de {nome} foi: {taxa_de_acerto}'
    print(msg)
    return(taxa_de_acerto)

def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

In [5]:
resultados = {}

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

A taxa de acerto de OneVsRest foi: 0.9257142857142858
A taxa de acerto de OneVsOne foi: 1.0
A taxa de acerto de MultinomialNB foi: 0.8366666666666667
A taxa de acerto de AdaBoostClassifier foi: 0.7528571428571429


In [6]:
for r in resultados.items():
    print(r)

(0.9257142857142858, OneVsRestClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)))
(1.0, OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)))
(0.8366666666666667, MultinomialNB())
(0.7528571428571429, AdaBoostClassifier())


In [7]:
maximo = max(resultados)
vencedor = resultados[maximo]

In [8]:
print("Vencerdor: ")
print(vencedor)

Vencerdor: 
OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0))


In [9]:
vencedor.fit(treino_dados, treino_marcacoes)

OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0))

In [11]:
teste_real(vencedor, validacao_dados, validacao_marcacoes)

Taxa de acerto do vencedor entre os algoritmos no mundo real: 100.0


In [12]:
acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

Taxa de acerto base: 82.608696


In [13]:
total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

Total de teste: 23


APRENDIZADO:
* Como classificar um elemento em 3 categorias
* Os algoritmos One vs Rest e One vs One
* A comparação entre os algoritmos OneVsRest, OneVsOne, MultinomialNB e AdaBoostClassifier